In [195]:
import numpy as np

from tempo.serve.metadata import ModelFramework, KubernetesOptions
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.utils import pipeline

In [196]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [5]:
!kubectl create ns production

namespace/production created


In [197]:
!kubectl apply -f ../tempo/tests/testdata/tempo-pipeline-rbac.yaml -n production

serviceaccount/tempo-pipeline unchanged
role.rbac.authorization.k8s.io/tempo-pipeline unchanged
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding unchanged


In [44]:
!helm upgrade --install redis bitnami/redis -n production \
    --set usePassword=false \
    --set master.service.type=LoadBalancer

Release "redis" does not exist. Installing it now.
NAME: redis
LAST DEPLOYED: Sun Mar  7 16:16:51 2021
NAMESPACE: production
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
** Please be patient while the chart is being deployed **

Redis can be accessed via port 6379 on the following DNS names from within your cluster:

redis-master.production.svc.cluster.local for read/write operations
redis-slave.production.svc.cluster.local for read-only operations



To connect to your Redis(TM) server:

1. Run a Redis(TM) pod that you can use as a client:
   kubectl run --namespace production redis-client --rm --tty -i --restart='Never' \
   
   --image docker.io/bitnami/redis:6.0.10-debian-10-r1 -- bash

2. Connect using the Redis(TM) CLI:
   redis-cli -h redis-master
   redis-cli -h redis-slave

To connect to your database from outside the cluster execute the following commands:

  NOTE: It may take a few minutes for the LoadBalancer IP to be available.
        Watch the status with: 'kubec

In [5]:
!kaggle datasets download -d uciml/default-of-credit-card-clients-dataset
!unzip -o default-of-credit-card-clients-dataset.zip

100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 3.41MB/s]
100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 3.40MB/s]
Archive:  default-of-credit-card-clients-dataset.zip
  inflating: UCI_Credit_Card.csv     


In [198]:
!mkdir -p artifacts/mab/
!mkdir -p artifacts/mab/route/
!mkdir -p artifacts/mab/feedback/

In [199]:
import pandas as pd
data = pd.read_csv('UCI_Credit_Card.csv')

In [200]:
target = 'default.payment.next.month'

In [201]:
import numpy as np
from sklearn.model_selection import train_test_split

OBSERVED_DATA = 15000
TRAIN_1 = 10000
TEST_1 = 5000

REST_DATA = 15000

RUN_DATA = 5000
ROUTE_DATA = 10000

# get features and target
X = data.loc[:, data.columns!=target].values
y = data[target].values

# observed/unobserved split
X_obs, X_rest, y_obs, y_rest = train_test_split(X, y, random_state=1, test_size=REST_DATA)

# observed split into train1/test1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_obs, y_obs, random_state=1, test_size=TEST_1)

# unobserved split into run/route
X_run, X_route, y_run, y_route = train_test_split(X_rest, y_rest, random_state=1, test_size=ROUTE_DATA)

# observed+run split into train2/test2
X_rest = np.vstack((X_run, X_route))
y_rest = np.hstack((y_run, y_route))

X_train2 = np.vstack((X_train1, X_test1))
X_test2 = X_run

y_train2 = np.hstack((y_train1, y_test1))
y_test2 = y_run

In [202]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train1, y_train1)

RandomForestClassifier(random_state=1)

In [203]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=1)
xgb.fit(X_train2, y_train2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [204]:
!mkdir -p artifacts/mab/sklearn/
!mkdir -p artifacts/mab/xgboost/

In [14]:
import joblib
joblib.dump(rf, 'artifacts/mab/sklearn/model.joblib')

['artifacts/mab/sklearn/model.joblib']

In [15]:
xgb.save_model('artifacts/mab/xgboost/model.bst')

In [234]:
import os

k8s_options = KubernetesOptions(namespace="production")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)

sklearn_model = Model(
        name="test-iris-sklearn",
        runtime=k8s_runtime,
        platform=ModelFramework.SKLearn,
        uri="gs://seldon-models/mab/sklearn",
        local_folder=os.getcwd()+"/artifacts/mab/sklearn")

xgboost_model = Model(
        name="test-iris-xgboost",
        runtime=k8s_runtime,
        platform=ModelFramework.XGBoost,
        uri="gs://seldon-models/mab/xgboost",
        local_folder=os.getcwd()+"/artifacts/mab/xgboost/")

In [ ]:
sklearn_model.upload()
xgboost_model.upload()

In [236]:
k8s_runtime_v2 = SeldonKubernetesRuntime(k8s_options=k8s_options, protocol=KFServingV2Protocol())

@pipeline(name="mab-pipeline",
          runtime=k8s_runtime_v2,
          uri="gs://seldon-models/mab/route",
          local_folder=os.getcwd()+"/artifacts/mab/route/",
          models=[sklearn_model, xgboost_model])
class MABRouter(object):

    def _init(self):
        self.n_branches = 2
        self.beta_params = [1 for _ in range(self.n_branches * 2)]
        
        import logging
        log = logging.getLogger(__name__)
        log.setLevel(10)
        self._log = log
        
        host = "redis-master"
        import os
        if os.environ.get("SELDON_LOCAL_ENVIRONMENT"):
            host = "localhost"
            
        self._log.info(f"Setting up redis with host {host}")
            
        import numpy as np
        self._np = np
        
        import redis
        self._rc = redis.Redis(host=host, port=6379)
        self._key = "seldon_deployment_predictor_model_1"
        
        if not self._rc.exists(self._key):
            models_beta_params = [1 for _ in range(self.n_branches * 2)]
            self._rc.lpush(self._key, *models_beta_params)
            
    @predictmethod
    def route(self, payload: np.ndarray) -> np.ndarray:

        if not hasattr(self, "_is_init") or not self._is_init:
            self._init()
            self._is_init = True
        
        models_beta_params = [float(i) for i in self._rc.lrange(self._key, 0, -1)]
        branch_values = [np.random.beta(a, b) for a, b in zip(*[iter(models_beta_params)] * 2)]
        selected_branch = np.argmax(branch_values)
        self._log.info(f"routing to branch: {selected_branch}")
        
        if selected_branch:
            return sklearn_model(payload)
        else:
            return xgboost_model(payload)

In [237]:
%env SELDON_LOCAL_ENVIRONMENT=LOCAL

mab_router = MABRouter()

env: SELDON_LOCAL_ENVIRONMENT=LOCAL


In [251]:
mab_router.route(payload=X_rest[0:1])

INFO:__main__:routing to branch: 0


array([0.0865844])

In [147]:
%%writefile artifacts/mab/route/conda.yaml
name: tempo
channels:
  - defaults
dependencies:
  - _libgcc_mutex=0.1=main
  - ca-certificates=2021.1.19=h06a4308_0
  - certifi=2020.12.5=py37h06a4308_0
  - ld_impl_linux-64=2.33.1=h53a641e_7
  - libedit=3.1.20191231=h14c3975_1
  - libffi=3.3=he6710b0_2
  - libgcc-ng=9.1.0=hdf63c60_0
  - libstdcxx-ng=9.1.0=hdf63c60_0
  - ncurses=6.2=he6710b0_1
  - openssl=1.1.1j=h27cfd23_0
  - pip=21.0.1=py37h06a4308_0
  - python=3.7.9=h7579374_0
  - readline=8.1=h27cfd23_0
  - setuptools=52.0.0=py37h06a4308_0
  - sqlite=3.33.0=h62c20be_0
  - tk=8.6.10=hbc83047_0
  - wheel=0.36.2=pyhd3eb1b0_0
  - xz=5.2.5=h7b6447c_0
  - zlib=1.2.11=h7b6447c_3
  - pip:
    - redis==3.5.3
    - websocket-client==0.58.0
    - mlops-tempo==0.1.0.dev4
    - mlserver==0.3.1.dev5
    - mlserver-tempo==0.3.1.dev5

Overwriting artifacts/mab/route/conda.yaml


In [253]:
# Currently needed as "save" doesn't fully work after sending a request
mab_router = MABRouter()

In [150]:
mab_router.save(save_env=True)

Packing environment at '/home/alejandro/miniconda3/envs/tempo-5c001ba8-2e2a-4842-ad38-7136bbf13f35' to '/home/alejandro/Programming/kubernetes/seldon/tempo/notebooks/artifacts/mab/route/environment.tar.gz'
[########################################] | 100% Completed | 34.3s


In [151]:
mab_router.upload()

In [254]:
mab_router.deploy()

INFO:tempo:deploying models for mab-pipeline
INFO:tempo:Found model test-iris-sklearn
INFO:tempo:Found model test-iris-xgboost


In [65]:
mab_router.wait_ready()

True

In [422]:
mab_router.remote(payload=X_rest[0:1])

array([[0.79, 0.21]])

In [292]:
@pipeline(name="mab-feedback",
          runtime=k8s_runtime_v2,
          local_folder=os.getcwd()+"/artifacts/mab/feedback/",
          conda_env="tempo",
          uri="gs://seldon-models/custom")
class MABFeedback(object):

    def _init(self):
        self.n_branches = 2
        self.beta_params = [1 for _ in range(self.n_branches * 2)]
        
        import logging
        log = logging.getLogger(__name__)
        log.setLevel(10)
        self._log = log
        
        host = "redis-master"
        import os
        if os.environ.get("SELDON_LOCAL_ENVIRONMENT"):
            host = "localhost"
        
        self._log.info(f"Setting up redis with host {host}")
            
        import numpy as np
        self._np = np
        
        import redis
        self._rc = redis.Redis(host=host, port=6379)
        self._key = "seldon_deployment_predictor_model_1"
        
        if not self._rc.exists(self._key):
            models_beta_params = [1 for _ in range(self.n_branches * 2)]
            self._log.info(f"Creating new key in redis with vals: {models_beta_params}")
            self._rc.lpush(self._key, *models_beta_params)
        else:
            self._log.info("Redis key already exists")

    @predictmethod
    def feedback(self, payload: np.ndarray, parameters: dict) -> np.ndarray:

        if not hasattr(self, "_is_init") or not self._is_init:
            self._init()
            self._is_init = True
            
        self._log.info(f"Feedback method with truth {payload} and parameters {parameters}")
                
        reward = parameters["reward"]
        routing = parameters["routing"]

        self._log.info(f"Sending feedback with route {routing} reward {reward}")
        
        # Currently only support 1 feedback at a time
        n_predictions = 1
        n_success = int(reward * n_predictions)
        n_failures = n_predictions - n_success
    
        self._log.info(f"n_success: {n_success}, n_failures: {n_failures}")

        # Non atomic, race condition op
        self._log.info(f"LINDEX key {self._key} on index {routing*2}")
        success_val = float(self._rc.lindex(self._key, int(routing*2)))
        self._rc.lset(self._key, int(routing*2), str(success_val + n_success))
        fail_val = float(self._rc.lindex(self._key, int(routing*2 + 1)))
        self._rc.lset(self._key, int(routing*2 + 1), str(fail_val + n_failures))
        
        return np.array([n_success, n_failures])
        

In [293]:
%env SELDON_LOCAL_ENVIRONMENT=LOCAL

mab_feedback = MABFeedback()

env: SELDON_LOCAL_ENVIRONMENT=LOCAL


In [294]:
X_rest[0:1]

array([[ 2.8590e+03,  5.0000e+05,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         4.0000e+01, -2.0000e+00, -2.0000e+00, -2.0000e+00, -2.0000e+00,
        -2.0000e+00, -2.0000e+00,  5.2550e+03,  7.2100e+02,  1.7252e+04,
         7.3880e+03,  6.0690e+03,  0.0000e+00,  7.2100e+02,  1.7252e+04,
         7.4210e+03,  6.0690e+03,  0.0000e+00,  0.0000e+00]])

In [295]:
mab_feedback.feedback(payload=X_rest[0:1], parameters={ "reward": 1, "routing": 0} )

INFO:__main__:Setting up redis with host localhost
INFO:__main__:Redis key already exists
INFO:__main__:Feedback method with truth [[ 2.8590e+03  5.0000e+05  1.0000e+00  1.0000e+00  1.0000e+00  4.0000e+01
  -2.0000e+00 -2.0000e+00 -2.0000e+00 -2.0000e+00 -2.0000e+00 -2.0000e+00
   5.2550e+03  7.2100e+02  1.7252e+04  7.3880e+03  6.0690e+03  0.0000e+00
   7.2100e+02  1.7252e+04  7.4210e+03  6.0690e+03  0.0000e+00  0.0000e+00]] and parameters {'reward': 1, 'routing': 0}
INFO:__main__:Sending feedback with route 0 reward 1
INFO:__main__:n_success: 1, n_failures: 0
INFO:__main__:LINDEX key seldon_deployment_predictor_model_1 on index 0


array([1, 0])

In [428]:
%env SELDON_LOCAL_ENVIRONMENT=LOCAL

mab_feedback = MABFeedback()

env: SELDON_LOCAL_ENVIRONMENT=LOCAL


In [429]:
mab_feedback.save(save_env=False)

In [298]:
mab_feedback.pipeline.upload()

In [299]:
mab_feedback.deploy()

INFO:tempo:deploying models for mab-feedback


In [ ]:
mab_feedback.wait_ready()

In [424]:
mab_feedback.remote(payload=X_rest[0:1], parameters={"reward":0.0,"routing":0} )

array([0, 1])